### Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

### Define the quadratic layer

In [14]:

class SimpleQuadratic(Layer):

    def __init__(self, units=32, activation=None):
        '''Initializes the class and sets up the internal variables'''
        super(SimpleQuadratic, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
    
    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        init_a = tf.random_normal_initializer()
        init_b = tf.random_normal_initializer()
        init_c = tf.zeros_initializer()

        init_a_val = init_a(shape=(input_shape[-1], self.units), dtype=tf.float32)
        init_b_val = init_b(shape=(input_shape[-1], self.units), dtype=tf.float32)
        init_c_val = init_c(shape=(self.units,), dtype=tf.float32)

        self.a = tf.Variable(name='krnel_1', initial_value=init_a_val, trainable=True)
        self.b = tf.Variable(name='krnel_1', initial_value=init_b_val, trainable=True)
        self.c = tf.Variable(name='bias', initial_value=init_c_val, trainable=True)

        super().build(input_shape)
    
   
    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        x_squared = tf.math.square(inputs)
        x_squared_a = tf.matmul(x_squared, self.a)
        x_times_b = tf.matmul(inputs, self.b)
        return self.activation(x_squared_a + x_times_b + self.c)

## Test Quadratic Layer on the MNIST Dataset
Expected performance >95%

In [16]:

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  SimpleQuadratic(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 12s 207us/sample - loss: 0.2700 - accuracy: 0.9200
Epoch 2/5
60000/60000 [==============================] - 12s 198us/sample - loss: 0.1336 - accuracy: 0.9595
Epoch 3/5
60000/60000 [==============================] - 12s 201us/sample - loss: 0.1005 - accuracy: 0.9696
Epoch 4/5
60000/60000 [==============================] - 12s 198us/sample - loss: 0.0843 - accuracy: 0.9736
Epoch 5/5
10000/10000 [==============================] - 1s 65us/sample - loss: 0.0796 - accuracy: 0.9756


[0.0796146826795768, 0.9756]

## Building Custom Loss Function with Keras API

In [ ]:
from tensorflow.keras import backend as k

def my_rmse(y_true, y_pred):
    error = y_true - y_pred
    square_error = k.square(error)
    mean_square_error = k.mean(square_error)
    root_mean_square_error = k.sqrt(mean_square_error)
    return root_mean_square_error